In [1]:
import pandas as pd
#Pull csv into pandas dataframe

data = pd.read_csv(r'C:\Users\Jordan\Documents\CodeKY\Capstone_Project_Repo\Data\pantheon_people.csv', delimiter = ",")
len(data)

11341

In [2]:
#drop unneeded columns
data = data.drop(columns=['en_curid','numlangs','countryCode','LAT','LON', 'TotalPageViews', 'L_star', 'StdDevPageViews', 'PageViewsEnglish', 'PageViewsNonEnglish', 'AverageViews' , 'HPI'                 ]) 
data.head(5)

,name,birthcity,birthstate,countryName,countryCode3,continentName,birthyear,gender,occupation,industry,domain
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,1809,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS
1,Aristotle,Stageira,NaN,Greece,GRC,Europe,-384,Male,PHILOSOPHER,PHILOSOPHY,HUMANITIES
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,1905,Female,WRITER,LANGUAGE,HUMANITIES
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,1970,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,1894,Male,WRITER,LANGUAGE,HUMANITIES


In [3]:
#replace the invalid vaues
#data2.loc[invalid_values.index, 'Birth_Year'] = pd.NA
#check to see if any values are returned
#invalid_values = data2[data2['Birth_Year'].astype(str).str.contains('[a-zA-Z!@#$%^&*(),.?":{}|<>]', regex=True, na=False)]
#print(invalid_values)

badyr = data['birthyear'].str.extract(r'^(\d{4})', expand=False)
badyr.head(10)
data['birthyear'] = pd.to_numeric(badyr)
data['birthyear'].dtype


dtype('float64')

In [4]:
#drop nas in year
data.dropna(subset=["birthyear"], inplace=True)
len(data)

10118

In [5]:
#drop where year is less than 1500 - prob won't have an accurate birthdate
data = data.drop(data[data['birthyear'] < 1500].index)
data.head(100)

,name,birthcity,birthstate,countryName,countryCode3,continentName,birthyear,gender,occupation,industry,domain
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,1809.0,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,1905.0,Female,WRITER,LANGUAGE,HUMANITIES
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,1970.0,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,1894.0,Male,WRITER,LANGUAGE,HUMANITIES
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,1932.0,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS
...,...,...,...,...,...,...,...,...,...,...,...
169,Ariel Sharon,Kfar Malal,NaN,Israel,ISR,Asia,1928.0,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS
170,Alexey Pazhitnov,Moscow,NaN,Russia,RUS,Europe,1956.0,Male,COMPUTER SCIENTIST,COMPUTER SCIENCE,SCIENCE & TECHNOLOGY
171,`Abdu'l-Bahá,Tehran,NaN,Iran,IRN,Asia,1844.0,Male,RELIGIOUS FIGURE,RELIGION,INSTITUTIONS
172,Akira Toriyama,Nagoya,NaN,JAPAN,JPN,Asia,1955.0,Male,COMIC ARTIST,DESIGN,ARTS


In [6]:
#convert birthyear to int
data['birthyear'] = data['birthyear'].astype('int64')

In [7]:
#determine the number of words in each name field
data["WordsCt"] = data["name"].apply(lambda n: len(n.split()))

In [8]:
#find and extract instances where names don't fit normal first/middle/last naming conventions
#def naming_exceptions(row):
 #   if ',' in row['name']:
  #          name2, othername = row['name'].split(',', 1)
   #         return pd.Series({'name2': name2, 'othername': othername})
    #else:
     #   return pd.Series({'name2': None, 'othername': None})
#data = pd.concat([data, data.apply(naming_exceptions, axis=1)], axis=1)

#def royalty (row):
 #   if '' in row['name']:
  #          name2, othername = row['name'].split(',', 1)
   #         return pd.Series({'name2': name2, 'othername': othername})
    #else:
     #   return pd.Series({'name2': None, 'othername': None})
#data = pd.concat([data, data.apply(naming_exceptions, axis=1)], axis=1)
    

In [9]:
# split items without a firstname into first, middle, last 

def split_names(row):
    if row['WordsCt'] == 2:
        first, last = row['name'].split()[:2]
        return pd.Series({'firstname': first, 'lastname': last})
    elif row['WordsCt'] == 3:
        first, middle, last = row['name'].split()[:3]
        return pd.Series({'firstname': first, 'middlename': middle, 'lastname': last})
    elif row['WordsCt'] == 1:
        first = row['name'].split()[:1]
        return pd.Series({'firstname': row['name'], 'middlename': None, 'lastname': None})        
    
    else:
        return pd.Series({'firstname': None, 'middlename': None, 'lastname': None})

# Apply the split_names function and concatenate the result to the original DataFrame
data = pd.concat([data, data.apply(split_names, axis=1)], axis=1)

data.head(20)

,name,birthcity,birthstate,countryName,countryCode3,continentName,birthyear,gender,occupation,industry,domain,WordsCt,firstname,lastname,middlename
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,1809,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS,2,Abraham,Lincoln,NaN
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,1905,Female,WRITER,LANGUAGE,HUMANITIES,2,Ayn,Rand,NaN
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,1970,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS,2,Andre,Agassi,NaN
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,1894,Male,WRITER,LANGUAGE,HUMANITIES,2,Aldous,Huxley,NaN
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,1932,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,2,Andrei,Tarkovsky,NaN
6,Arthur Schopenhauer,Gdańsk,NaN,POLAND,POL,Europe,1788,Male,PHILOSOPHER,PHILOSOPHY,HUMANITIES,2,Arthur,Schopenhauer,NaN
7,Albert Einstein,Ulm,NaN,Germany,DEU,Europe,1879,Male,PHYSICIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY,2,Albert,Einstein,NaN
9,Alfred Hitchcock,Leytonstone,NaN,UNITED KINGDOM,GBR,Europe,1899,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,2,Alfred,Hitchcock,NaN
11,Alfred Nobel,Stockholm,NaN,SWEDEN,SWE,Europe,1833,Male,CHEMIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY,2,Alfred,Nobel,NaN
12,Alexander Graham Bell,Edinburgh,NaN,UNITED KINGDOM,GBR,Europe,1847,Male,INVENTOR,INVENTION,SCIENCE & TECHNOLOGY,3,Alexander,Bell,Graham


In [10]:
#drop rows that could not be split out
data = data.dropna(subset=['firstname'])
len(data)

8867

In [11]:
#create people look-up 
#get the first 3 letters of the first and last name
#cast year to a string and concatenate to create a value to join with other dataset
data['First3'] = data['firstname'].str[:3]
data['Last3'] = data['lastname'].str[:3]
data['birthyear2'] = data['birthyear'].astype(str)
data['People_Lookup'] = data['First3'] + data['Last3'] + data['birthyear2']
data = data.drop(columns=['WordsCt', 'First3', 'Last3', 'birthyear2', 'birthyear'])

data.head(10)

,name,birthcity,birthstate,countryName,countryCode3,continentName,gender,occupation,industry,domain,firstname,lastname,middlename,People_Lookup
0,Abraham Lincoln,Hodgenville,KY,UNITED STATES,USA,North America,Male,POLITICIAN,GOVERNMENT,INSTITUTIONS,Abraham,Lincoln,NaN,AbrLin1809
2,Ayn Rand,Saint Petersburg,NaN,Russia,RUS,Europe,Female,WRITER,LANGUAGE,HUMANITIES,Ayn,Rand,NaN,AynRan1905
3,Andre Agassi,Las Vegas,NV,UNITED STATES,USA,North America,Male,TENNIS PLAYER,INDIVIDUAL SPORTS,SPORTS,Andre,Agassi,NaN,AndAga1970
4,Aldous Huxley,Godalming,NaN,UNITED KINGDOM,GBR,Europe,Male,WRITER,LANGUAGE,HUMANITIES,Aldous,Huxley,NaN,AldHux1894
5,Andrei Tarkovsky,Zavrazhye,NaN,Russia,RUS,Europe,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,Andrei,Tarkovsky,NaN,AndTar1932
6,Arthur Schopenhauer,Gdańsk,NaN,POLAND,POL,Europe,Male,PHILOSOPHER,PHILOSOPHY,HUMANITIES,Arthur,Schopenhauer,NaN,ArtSch1788
7,Albert Einstein,Ulm,NaN,Germany,DEU,Europe,Male,PHYSICIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY,Albert,Einstein,NaN,AlbEin1879
9,Alfred Hitchcock,Leytonstone,NaN,UNITED KINGDOM,GBR,Europe,Male,FILM DIRECTOR,FILM AND THEATRE,ARTS,Alfred,Hitchcock,NaN,AlfHit1899
11,Alfred Nobel,Stockholm,NaN,SWEDEN,SWE,Europe,Male,CHEMIST,NATURAL SCIENCES,SCIENCE & TECHNOLOGY,Alfred,Nobel,NaN,AlfNob1833
12,Alexander Graham Bell,Edinburgh,NaN,UNITED KINGDOM,GBR,Europe,Male,INVENTOR,INVENTION,SCIENCE & TECHNOLOGY,Alexander,Bell,Graham,AleBel1847


In [12]:
#write to table in SQLlite db

import sqlite3
connection = sqlite3.connect('Zodiac_Analysis.db')
data.to_sql('Pantheon_People_Import', connection, if_exists='replace')

8867